In [1]:
import torch
from transformers import BertTokenizer
#from clinical_bert_similarity import ClinicalBertSimilarity

# Load the saved model
model_path = '/kaggle/input/sick-dataset-model/sick_model.pth'  # Update with the path where you saved the model
loaded_model = ClinicalBertSimilarity(device='cuda')  # Assuming you want to load the model on GPU
loaded_model.regressor_net.load_state_dict(torch.load(model_path))
#loaded_model.to('cuda')  # Move the model to GPU if available

# Define the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input sentences
s1 = input("Enter the first sentence: ").lower()
s2 = input("Enter the second sentence: ").lower()

# Predict similarity score
predictions = loaded_model.predict([(s1, s2)])
print("Similarity score is: ", predictions[0])


NameError: name 'ClinicalBertSimilarity' is not defined

In [2]:
!pip install pytorch-transformers
!pip install wandb
!pip install torch torchvision torchaudio



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 79.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.29.165 which is incompatible.


In [ ]:
import logging
from pytorch_transformers import BertTokenizer, BertConfig, BertModel
from pytorch_transformers.modeling_utils import CONFIG_NAME
from torch import nn
from scipy.stats import spearmanr
from pytorch_transformers.file_utils import url_to_filename, http_get
import tarfile
import tempfile
import torch.optim as optim
import torch 
import os
from scipy.stats import pearsonr, spearmanr
from tqdm import tqdm
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer
from pytorch_transformers.modeling_utils import WEIGHTS_NAME, CONFIG_NAME
from pytorch_transformers.modeling_bert import BertPreTrainedModel, BertConfig, BertModel
import os, tarfile, urllib, tempfile, shutil
#from pytorch_transformers.file_utils import url_to_filename, http_get
import wandb
wandb.init(project='Semantic_textual_similarity', entity='bluehoax')


try:
    from torch.hub import _get_torch_home
    torch_cache_home = _get_torch_home()
except ImportError:
    torch_cache_home = os.path.expanduser(
        os.getenv('TORCH_HOME', os.path.join(
            os.getenv('XDG_CACHE_HOME', '~/.cache'), 'torch')))

default_cache_path = os.path.join(torch_cache_home, 'semantic_text_similarity')

MODEL_URL = {
    'clinical-bert-similarity': 'https://github.com/AndriyMulyar/semantic-text-similarity/releases/download/v1.0.0/clinical_bert_similarity.tar.gz',
    'web-bert-similarity': 'https://github.com/AndriyMulyar/semantic-text-similarity/releases/download/v1.0.0/web_bert_similarity.tar.gz'
}
def get_model_path(model_name: str):

    if model_name in MODEL_URL:
        model_url = MODEL_URL[model_name]
    else:
        return model_name
    model_url_hash = url_to_filename(model_url)

    model_path = os.path.join(default_cache_path, model_url_hash)

    if os.path.exists(model_path):
        return model_path
    else:
        os.makedirs(model_path)
        with tempfile.NamedTemporaryFile() as temp_file:
            print("Downloading model: %s from %s" % (model_name, model_url))
            try:
                http_get(model_url, temp_file)
                tar = tarfile.open(temp_file.name)
            except BaseException as exc:
                print("Failed to download model: %s"% model_name)
                os.rmdir(model_path)
                raise exc


            temp_file.flush()
            temp_file.seek(0)
            tar.extractall(model_path)

            return model_path

def _truncate_seq_pair(tokens_a, tokens_b, max_length: int):
    """
    Copied exactly from: https://github.com/huggingface/pytorch-pretrained-BERT/blob/78462aad6113d50063d8251e27dbaadb7f44fbf0/examples/extract_features.py#L150
    Truncates a sequence pair in place to the maximum length.
    """

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def bert_sentence_pair_preprocessing(data: list, tokenizer: BertTokenizer, max_sequence_length=128):
    """
    Pre-processes an array of sentence pairs for input into bert. Sentence pairs are expected to be processed
    as given in data.py.

    Each sentence pair is tokenized and concatenated together by the [SEP] token for input into BERT

    :return: three tensors: [data_size, input_ids], [data_size, token_type_ids], [data_size, attention_mask]
    """

    max_bert_input_length = 0
    for sentence_pair in data:

        sentence_1_tokenized, sentence_2_tokenized = tokenizer.tokenize(sentence_pair['sentence_1']), tokenizer.tokenize(sentence_pair['sentence_2'])
        _truncate_seq_pair(sentence_1_tokenized, sentence_2_tokenized, max_sequence_length - 3) #accounting for positioning tokens


        max_bert_input_length = max(max_bert_input_length, len(sentence_1_tokenized) + len(sentence_2_tokenized) + 3)
        sentence_pair['sentence_1_tokenized'] = sentence_1_tokenized
        sentence_pair['sentence_2_tokenized'] = sentence_2_tokenized

    dataset_input_ids = torch.empty((len(data), max_bert_input_length), dtype=torch.long)
    dataset_token_type_ids = torch.empty((len(data), max_bert_input_length), dtype=torch.long)
    dataset_attention_masks = torch.empty((len(data), max_bert_input_length), dtype=torch.long)
    dataset_scores = torch.empty((len(data), 1), dtype=torch.float)

    for idx, sentence_pair in enumerate(data):
        tokens = []
        input_type_ids = []

        tokens.append("[CLS]")
        input_type_ids.append(0)
        for token in sentence_pair['sentence_1_tokenized']:
            tokens.append(token)
            input_type_ids.append(0)
        tokens.append("[SEP]")
        input_type_ids.append(0)

        for token in sentence_pair['sentence_2_tokenized']:
            tokens.append(token)
            input_type_ids.append(1)
        tokens.append("[SEP]")
        input_type_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        attention_masks = [1] * len(input_ids)
        while len(input_ids) < max_bert_input_length:
            input_ids.append(0)
            attention_masks.append(0)
            input_type_ids.append(0)


        dataset_input_ids[idx] = torch.tensor(input_ids, dtype=torch.long)
        dataset_token_type_ids[idx] = torch.tensor(input_type_ids, dtype=torch.long)
        dataset_attention_masks[idx] = torch.tensor(attention_masks, dtype=torch.long)
        if 'similarity' not in sentence_pair or sentence_pair['similarity'] is None:
            dataset_scores[idx] = torch.tensor(float('nan'), dtype=torch.float)
        else:
            dataset_scores[idx] = torch.tensor(sentence_pair['similarity'], dtype=torch.float)


    return dataset_input_ids, dataset_token_type_ids, dataset_attention_masks, dataset_scores
class BertSimilarityRegressor(BertPreTrainedModel):
    def __init__(self, bert_model_config: BertConfig):
        super(BertSimilarityRegressor, self).__init__(bert_model_config)
        self.bert = BertModel(bert_model_config)
        linear_size = bert_model_config.hidden_size
        if bert_model_config.pretrained_config_archive_map['additional_features'] is not None:
            linear_size += bert_model_config.pretrained_config_archive_map['additional_features']

        self.regression = nn.Sequential(
            nn.Dropout(p=bert_model_config.hidden_dropout_prob),
            nn.Linear(linear_size, 1)
        )

        self.init_weights()

    def init_weights(self):
        # Initialize weights for linear layer
        self.regression.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(self, input_ids, token_type_ids, attention_masks, additional_features=None):
        """
        Feed forward network with one hidden layer.
        :param input_ids:
        :param token_type_ids:
        :param attention_masks:
        :return:
        """
        
        _, pooled_output = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_masks)

        if additional_features is not None:
            pooled_output = torch.cat((pooled_output, additional_features), dim=1)

        return self.regression(pooled_output)


class BertSimilarity():
    """
    A class implementing the training and evaluation of a fine-tuned BERT model for sentence pair similarity.
    """

    def __init__(self, args=None, device='cuda', bert_model_path='bert-base-uncased', batch_size=10, learning_rate=5e-5, weight_decay=0, additional_features=None):
        if args is not None:
            self.args = vars(args)

        assert device in ['cuda', 'cpu']

        if not args:
            self.args = {}
            self.args['bert_model_path'] = bert_model_path
            self.args['device'] = device
            self.args['learning_rate'] = learning_rate
            self.args['weight_decay'] = weight_decay
            self.args['batch_size'] = batch_size

        self.log = logging.getLogger()

        self.bert_tokenizer = BertTokenizer.from_pretrained(self.args['bert_model_path'])
        if os.path.exists(self.args['bert_model_path']):
            if os.path.exists(os.path.join(self.args['bert_model_path'], CONFIG_NAME)):
                config = BertConfig.from_json_file(os.path.join(self.args['bert_model_path'], CONFIG_NAME))
            elif os.path.exists(os.path.join(self.args['bert_model_path'], 'bert_config.json')):
                config = BertConfig.from_json_file(os.path.join(self.args['bert_model_path'], 'bert_config.json'))
            else:
                raise ValueError("Cannot find a configuration for the BERT model you are attempting to load.")

        self.loss_function = torch.nn.MSELoss()

        config.pretrained_config_archive_map['additional_features'] = additional_features

        self.regressor_net = BertSimilarityRegressor.from_pretrained(self.args['bert_model_path'], config=config)
        self.optimizer = torch.optim.Adam(
            self.regressor_net.parameters(),
            weight_decay=self.args['weight_decay'],
            lr=self.args['learning_rate']
        )
        self.log.info('Initialized BertSentencePairSimilarity model from %s' % self.args['bert_model_path'])

    def predict(self, data: list, additional_features=None, return_predictions=True):
        """
        Given a list of sentence pair instances, makes predictions
        :param data:
        :param return predictions or log to model directory
        :return:
        """

        self.regressor_net.to(device=self.args['device'])
        self.regressor_net.eval()

        with torch.no_grad():

            if data is not None and isinstance(data, list):
                if isinstance(data[0], dict):
                    input_ids_eval, token_type_ids_eval, attention_masks_eval, correct_scores_eval = bert_sentence_pair_preprocessing(
                        data, self.bert_tokenizer)
                elif isinstance(data[0], tuple):
                    input_ids_eval, token_type_ids_eval, attention_masks_eval, correct_scores_eval = bert_sentence_pair_preprocessing(
                        [{'sentence_1': s1, 'sentence_2': s2} for s1, s2 in data], self.bert_tokenizer)
                else:
                    raise ValueError("Data must be a list of sentence pair tuples")

            predictions = torch.empty_like(correct_scores_eval)
            for i in range(0, input_ids_eval.shape[0], self.args['batch_size']):
                input_id_eval = input_ids_eval[i:i + self.args['batch_size']].to(device=self.args['device'])
                token_type_id_eval = token_type_ids_eval[i:i + self.args['batch_size']].to(device=self.args['device'])
                attention_mask_eval = attention_masks_eval[i:i + self.args['batch_size']].to(device=self.args['device'])

                if additional_features is not None:
                    additional_feature = additional_features[i:i + self.args['batch_size']].to(device=self.args['device'])
                else:
                    additional_feature = None

                predicted_score = self.regressor_net(input_id_eval, token_type_id_eval, attention_mask_eval, additional_features=additional_feature)
                predictions[i:i + self.args['batch_size']] = predicted_score

            if all(torch.isnan(correct_scores_eval)) or return_predictions:  # No correct score labels are present, return the predictions
                return predictions.cpu().view(-1).numpy()
            else:
                self.log.info('Evaluating on Epoch %i' % (self.epoch))

                # Calculate Spearman correlation coefficient
                spearman_corr, _ = spearmanr(predictions.cpu().view(-1).numpy(), correct_scores_eval.view(-1).numpy())
                self.log.info("Spearman: %f" % spearman_corr)

                with open(os.path.join(self.args['model_directory'], "eval_%s.csv" % self.epoch), 'w') as eval_results:
                    eval_results.write("sentence_1\tsentence_2\tannotator_score\tpredicated_score\n")

                    for idx, row in enumerate(torch.cat((correct_scores_eval, predictions.cpu()), dim=1)):
                        eval_results.write(
                            "%s\t%s\t%f\t%f\n" % (data[idx]['sentence_1'], data[idx]['sentence_2'], row[0].item(), row[1].item()))
                    eval_results.write("Spearman\t%f" % spearman_corr)

                self.log.info("Spearman: %f" % spearman_corr)
        self.regressor_net.train()

def _truncate_seq_pair(tokens_a, tokens_b, max_length: int):
    """
    Copied exactly from: https://github.com/huggingface/pytorch-pretrained-BERT/blob/78462aad6113d50063d8251e27dbaadb7f44fbf0/examples/extract_features.py#L150
    Truncates a sequence pair in place to the maximum length.
    """

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()
class STSBDataset(Dataset):
    def __init__(self, data, tokenizer, max_sequence_length=64):
        self.data = data
        self.tokenizer = tokenizer
        self.max_sequence_length = max_sequence_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        input_ids, token_type_ids, attention_masks, labels = self.tokenize_and_convert_to_tensor(sample['sentence_1'], sample['sentence_2'], sample['similarity'])
        
        # Convert lists to tensors
        input_ids = torch.tensor(input_ids)
        token_type_ids = torch.tensor(token_type_ids)
        attention_masks = torch.tensor(attention_masks)
        labels = torch.tensor(labels)
        
        return input_ids, token_type_ids, attention_masks, labels

    def tokenize_and_convert_to_tensor(self, sentence_1, sentence_2, similarity_score):
        sentence_1_tokenized = self.tokenizer.tokenize(sentence_1)
        sentence_2_tokenized = self.tokenizer.tokenize(sentence_2)

        # Truncate sequences if they exceed max_sequence_length - 3 for [CLS], [SEP], [SEP]
        _truncate_seq_pair(sentence_1_tokenized, sentence_2_tokenized, self.max_sequence_length - 3)

        # Add [CLS] and [SEP] tokens, and generate input_ids, token_type_ids, and attention_masks
        input_ids = self.tokenizer.convert_tokens_to_ids(['[CLS]'] + sentence_1_tokenized + ['[SEP]'] + sentence_2_tokenized + ['[SEP]'])
        token_type_ids = [0] * (len(sentence_1_tokenized) + 2) + [1] * (len(sentence_2_tokenized) + 1)
        attention_masks = [1] * len(input_ids)

        # Pad sequences to max_sequence_length
        input_ids += [0] * (self.max_sequence_length - len(input_ids))
        token_type_ids += [0] * (self.max_sequence_length - len(token_type_ids))
        attention_masks += [0] * (self.max_sequence_length - len(attention_masks))

        labels = similarity_score  # Assuming similarity_score is a float
        return input_ids, token_type_ids, attention_masks, labels

def load_sts_b_data(tokenizer):
    sick_file_path = '/kaggle/input/sickdataset/SICK.txt'
    if not os.path.exists(sick_file_path):
        raise FileNotFoundError('Cannot find SICK dataset')

    with open(sick_file_path, 'r', encoding='utf-8') as file:
        file.readline()  # Skip header line
        sick_data = file.readlines()

    total_samples = len(sick_data)
    train_size = int(0.8 * total_samples)
    dev_size = int(0.1 * total_samples)

    train_data = sick_data[:train_size]
    dev_data = sick_data[train_size:train_size + dev_size]
    test_data = sick_data[train_size + dev_size:]

    def process_data(data):
        processed_data = []
        for line in data:
            parts = line.strip().split('\t')
            pair_ID, sentence_A, sentence_B, _, relatedness_score, _, _, _, _, _, l, k = parts

            processed_data.append({
                'sentence_1': sentence_A,
                'sentence_2': sentence_B,
                'similarity': float(relatedness_score)
            })
        return processed_data

    train_data = process_data(train_data)
    dev_data = process_data(dev_data)
    test_data = process_data(test_data)

    return train_data, dev_data, test_data

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the data
train_data, dev_data, test_data = load_sts_b_data(tokenizer)

# Create DataLoader instances for train, dev, and test sets
batch_size = 10
train_loader = DataLoader(STSBDataset(train_data, tokenizer), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(STSBDataset(test_data,tokenizer),batch_size=batch_size)
class ClinicalBertSimilarity(BertSimilarity):
    def __init__(self, device='cuda', batch_size=10, model_name="web-bert-similarity"):
        model_path = get_model_path(model_name)
        super().__init__(device=device, batch_size=batch_size, bert_model_path=model_path)
        self.device = torch.device(device)  # Convert device to a Torch device object
        self.regressor_net.to(self.device)   # Move model to CUDA device

    def train_model(self, train_loader, num_epochs=10, validation_loader=None):
        # Set the model to training mode
        self.regressor_net.train()

        # Define loss function and optimizer
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.regressor_net.parameters(), lr=self.args['learning_rate'], weight_decay=self.args['weight_decay'])

        for epoch in range(num_epochs):
            total_loss = 0.0
            tqdm_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
            for i, batch in enumerate(tqdm_bar):
                input_ids, token_type_ids, attention_masks, labels = batch
                # Move data to device (CUDA)
                input_ids, token_type_ids, attention_masks, labels = input_ids.to(self.device), token_type_ids.to(self.device), attention_masks.to(self.device), labels.to(self.device)

                # Zero the gradients
                optimizer.zero_grad() 
                outputs = self.regressor_net(input_ids, token_type_ids, attention_masks)

                labels = labels.unsqueeze(1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)
            wandb.log({"Training Loss": avg_loss})
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader)}")

    
    def predict(self, data: list, actual_scores=None, additional_features=None, return_predictions=True):
        """
        Given a list of sentence pair instances, makes predictions
        :param data: List of sentence pairs
        :param actual_scores: Actual similarity scores corresponding to the data (optional)
        :param additional_features: Additional features to include in prediction (optional)
        :param return_predictions: Whether to return predictions or log to model directory
        :return: Predictions or Spearman correlation coefficient (if actual_scores provided)
        """

        self.regressor_net.to(device=self.args['device'])
        self.regressor_net.eval()

        with torch.no_grad():
            if data is not None and isinstance(data, list):
                if isinstance(data[0], dict):
                    input_ids_eval, token_type_ids_eval, attention_masks_eval, correct_scores_eval = bert_sentence_pair_preprocessing(
                        data, self.bert_tokenizer)
                elif isinstance(data[0], tuple):
                    input_ids_eval, token_type_ids_eval, attention_masks_eval, correct_scores_eval = bert_sentence_pair_preprocessing(
                        [{'sentence_1': s1, 'sentence_2': s2} for s1, s2 in data], self.bert_tokenizer)
                else:
                    raise ValueError("Data must be a list of sentence pair tuples")

            predictions = torch.empty_like(correct_scores_eval)
            for i in range(0, input_ids_eval.shape[0], self.args['batch_size']):
                input_id_eval = input_ids_eval[i:i + self.args['batch_size']].to(device=self.args['device'])
                token_type_id_eval = token_type_ids_eval[i:i + self.args['batch_size']].to(device=self.args['device'])
                attention_mask_eval = attention_masks_eval[i:i + self.args['batch_size']].to(device=self.args['device'])

                if additional_features is not None:
                    additional_feature = additional_features[i:i + self.args['batch_size']].to(device=self.args['device'])
                else:
                    additional_feature = None

                predicted_score = self.regressor_net(input_id_eval, token_type_id_eval, attention_mask_eval, additional_features=additional_feature)
                predictions[i:i + self.args['batch_size']] = predicted_score

            if all(torch.isnan(correct_scores_eval)) or return_predictions:  
                # No correct score labels are present or return predictions is requested, return the predictions
                return predictions.cpu().view(-1).numpy()
#             else:
#                 # Calculate Spearman correlation coefficient if actual scores are provided
#                 spearman_corr, _ = spearmanr(predictions.cpu().view(-1).numpy(), actual_scores.view(-1).numpy())
#                 return spearman_corr

    def save_model(self, filename):
        torch.save(self.regressor_net.state_dict(), filename)
device="cuda"
batch_size=10
model = ClinicalBertSimilarity(device, batch_size)

# Train the model
model.train_model(train_loader,5)


# Use the predict function to get predictions
predictions = model.predict(test_data)
predictions_dev=model.predict(dev_data)
# Extract the actual similarity scores from the test data
actual_scores = [sample['similarity'] for sample in test_data]
actual_scores_dev=[sample['similarity'] for sample in dev_data]
# Calculate Spearman correlation
spearman_corr, _ = spearmanr(predictions, actual_scores)
print("Spearman correlation on test:", spearman_corr)

# Optionally, you can also calculate Pearson correlation
pearson_corr, _ = pearsonr(predictions, actual_scores)
print("Pearson correlation on test:", pearson_corr)

spearman_corr, _ = spearmanr(predictions_dev, actual_scores_dev)
pearson_corr, _ = pearsonr(predictions_dev, actual_scores_dev)
print("spearman correlation on dev:",spearman_corr)
print("pearson correlation on dev:",pearson_corr)
# Save the trained model
model.save_model('sick_model.pth')
print("Model saved successfully")


    

s1 = input("Enter the first sentence: ").lower()
s2 = input("Enter the second sentence: ").lower()
predictions = model.predict([(s1, s2)])
print("Similarity score is: ", predictions[0])


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


100%|██████████| 231508/231508 [00:00<00:00, 4148271.43B/s]


Epoch 1/10: 100%|██████████| 788/788 [01:17<00:00, 10.23it/s]


...

Epoch [8/10], Loss: 0.08630332436299112


Epoch 9/10: 100%|██████████| 788/788 [01:15<00:00, 10.38it/s]


Epoch [9/10], Loss: 0.07313234795612959


Epoch 10/10: 100%|██████████| 788/788 [01:15<00:00, 10.38it/s]


Epoch [10/10], Loss: 0.07306513479416764
Spearman correlation on test: 0.9017261361350308
Pearson correlation on test: 0.9254133815962392
spearman correlation on dev: 0.7541430467708301
pearson correlation on dev: 0.7736874106319803
Model saved successfully
